# Projet de fin de semestre : SPAM CLASSIFIEUR

## Réalisé par :

- TAIBI      Abdessetar 191932027573 GROUPE 1

- DJENANE    Nihad      191931040689 GROUPE 1

- TALABOULMA Roumaissa  191932021195 GROUPE 1

- M'BAREK    Lydia      181831064011 GROUPE 1





L'objectif de ce projet est de développer un système de détection de spam

Dans ce projet, nous avons utilise un ensemble d'e-mails étiquetés comme étant du spam 
et nous avons utilise différentes approches pour développer notre classifieur de détection de spam. 

Les performances de chaque approche seront comparées pour évaluer leur efficacité et déterminer la meilleure approche pour la détection de spam

# Importation des librairies necessaires au travail

Nous utilisons pour dans ce projet les librairies suivantes :

- **NLTK** : Librairie de traitement de langague, contient des méthodes de traitement de text (stemming, tokenization, ...). Nous utilisons *SnowballStemmer* pour réduire les tokens en à leurs radicaux.

- **sklearn** : Nous utilisons cette librairie pour entrainer des algorithmes d'apprentissage machine comme les SVM et Reseaux de neurones Nous utilisons les metriques proposées par cette librairie pour comparer les résultats obtenus par chaque modèle.


In [64]:
from nltk.stem import SnowballStemmer
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
import numpy as np
import pandas as pd 
import os
import re

# Étape 1 : préparation des données (Preprocessing)
Avant de développer le classifieur de détection de spam, une préparation des données sera effectuée, comprenant la collecte des e-mails, leur nettoyage et leur transformation en un format utilisable pour l'entraînement et la validation des modèles de classification.

1. Normalisation des emails : 

In [65]:
def clean_mail(mail):
    # Convertir en minuscules
    mail = mail.lower()
    # Supprimer les en-têtes et les pieds de page de l'e-mail
    mail = re.sub(r'^.*?(\n\n|\r\n\r\n)', '', mail, flags=re.S)
    # supprimer les espaces et saut de ligne
    mail = re.sub(r'\s+', ' ', mail)
    # Supprimer les balises HTML
    mail = re.sub(r'<.*?>', ' ', mail)
    # Normaliser les URL en remplaçant par "httpaddr"
    mail  = re.sub(r'(http|https)://[^\s]+', "httpaddr", mail)
    # Normaliser les adresses e-mail en remplaçant par "emailaddr"
    mail  = re.sub(r'\b[\w\.-]+@[\w\.-]+\.\w{2,}\b', "emailaddr", mail)
    # Remplacer le symbole dollar ($) par "dollar"
    mail = re.sub('\$', 'dollar', mail)
    # Remplacer tous les nombres par "number"
    mail = re.sub('\d+', 'number', mail)
    # Séparer les mots de l'e-mail en une liste
    mail_words = mail.split()
    # Initialiser le stemmer et le stopwords pour normaliser les mots
    stemmer = SnowballStemmer("english")
    # Normaliser chaque mot de l'e-mail en sa forme radicale 
    mail_words = [stemmer.stem(word) for word in mail_words]
    # Joindre les mots normalisés pour reformer l'e-mail
    mail_cleaned = " ".join(mail_words)
    # Supprimer les caractères spéciaux et les espaces supplémentaires
    mail_cleaned = re.sub(r'[^\w\s]|_', ' ', mail_cleaned)
    mail_cleaned = re.sub(r'\s+', ' ', mail_cleaned).strip()
    # Renvoyer l'e-mail nettoyé
    return mail_cleaned

In [66]:
def get_data(path):
    # Création d'une liste vide pour stocker les données
    data = []
    # Récupération des noms de tous les fichiers présents dans le chemin spécifié
    files = os.listdir(path)    
    # Parcours de tous les fichiers présents dans le répertoire
    for file in files:
        if file == "cmds":
            pass
        # Ouverture du fichier en mode lecture 
        processed_file = open(path +"/"+ file, encoding="ISO-8859-1")        
        # Lecture du contenu du fichier
        words_list = processed_file.read()        
        # Ajout du contenu du fichier à la liste de données
        data.append(words_list)        
        # Fermeture du fichier
        processed_file.close()    
    # Renvoi de la liste de données
    return data

In [67]:
spam_mails_path = 'Data/spam_2/spam_2'
hard_ham_mails_path = 'Data/hard_ham/hard_ham'
easy_ham_mails_path = 'Data/easy_ham/easy_ham'

In [68]:
# donnes
data_spam_mails = get_data(spam_mails_path)
data_hard_ham_mails = get_data(hard_ham_mails_path)
data_easy_ham_mails = get_data(easy_ham_mails_path)

In [69]:
def clean_all_mails(data_mail):
    # Création d'une liste vide pour stocker les mails nettoyés
    mails_clean = []
    # Parcours de la liste de mails 
    for mail in data_mail:
        # nettoyer le mail et l'ajouter à la liste "mails_clean"
        mails_clean.append(clean_mail(mail))
    return mails_clean

In [70]:
# nettoyer les emails spam
spam_mails_clean = clean_all_mails(data_spam_mails)
print(len(spam_mails_clean))

1396


In [71]:
# nettoyer les emails ham
hard_ham_mails_clean = clean_all_mails(data_hard_ham_mails)
easy_ham_mails_clean = clean_all_mails(data_easy_ham_mails)

In [72]:
# Visualiser les donnes spam
spam = pd.DataFrame(spam_mails_clean)
spam.head

<bound method NDFrame.head of                                                       0
0     greetings you are receiv this letter becaus yo...
1     the need for safeti is real in number you migh...
2     bonus fat absorb as seen on tv includ free wit...
3     bonus fat absorb as seen on tv includ free wit...
4     govern grant e book number edition just dollar...
...                                                 ...
1391  want to be your own boss nbsp train now with s...
1392  this is a multi part messag in mime format nex...
1393  dear subscriber if i could show you a way to g...
1394  mid summ custom appreci sale to express our ap...
1395  attn sir madan strict confidential i am pleas ...

[1396 rows x 1 columns]>

2.  Construction du vocabulaire 

In [73]:
# Joindre tous les mails nettoyés en une seule chaîne de caractères
all_spam_mails = " ".join(spam_mails_clean)
# obtenir tous les mots de tous les emails spam
all_words = all_spam_mails.split()  
print(len(all_words))

558060


In [74]:
print(len(all_spam_mails))

4010965


In [75]:
def get_vocab_from_file():
    with open("vocab.txt", "r") as f:
        content = f.read()
        vocab_list = content.split()
    return vocab_list

def build_and_save_vocab(all_words_mails, k):
   # Compter le nombre d'occurrences de chaque mot
    all_word_counts = np.unique(all_words_mails, return_counts=True)
    # Trouver les indices des mots qui apparaissent k fois ou plus
    frequent_word_indices = np.where(all_word_counts[1] >= k)[0]
    # Extraire les mots fréquents et les enregistrer dans le fichier "vocab.txt"
    frequent_words = all_word_counts[0][frequent_word_indices]
    np.savetxt("vocab.txt", frequent_words, fmt="%s")
    return frequent_words


In [76]:
# construire une liste de vocabulaire, nous ajoutons les mots qui se répètent au moins K fois
k = 10
vocab_list = build_and_save_vocab(all_words ,k)

In [77]:
print(len(vocab_list))

3740


In [78]:
def indexation_mail(all_mails_clean):
    all_mails_index = []# Liste pour stocker les index des mails
    for mail in all_mails_clean:
        mail_index = [] # Liste pour stocker les index des mots dans un mail
        for word in mail.split():
            index_word = np.searchsorted(vocab_list, word)  # Recherche de l'index du mot dans le vocabulaire
            if index_word < len(vocab_list) and vocab_list[index_word] == word :
                   mail_index.append(index_word) # Ajout de l'index du mot à la liste des index du mail si le mot existe dans vocab_list
        all_mails_index.append(mail_index)  # Ajout le mail indexe à la liste de tous les mails
    return all_mails_index
        

In [79]:
# indexer les mails
spam_mails_index = indexation_mail(spam_mails_clean)
hard_ham_mails_index = indexation_mail(hard_ham_mails_clean)
easy_ham_mails_index = indexation_mail(easy_ham_mails_clean)


In [94]:
y_spam = np.ones(len(spam_mails_index) , dtype=int)
y_ham = np.zeros(len(hard_ham_mails_index) + len(easy_ham_mails_index) , dtype=int)
# Concaténation des tableaux 'spam_mails_index', 'hard_ham_mails_index' et 'easy_ham_mails_index' dans 'all_mails_index'
all_mails_index = np.concatenate((spam_mails_index , hard_ham_mails_index , easy_ham_mails_index) , axis=0)
# Concaténation des tableaux 'y_spam' et 'y_ham' dans 'y'
y = np.concatenate((y_spam , y_ham),axis=0)
m = len(all_mails_index)

In [100]:
X = np.zeros((m, len(vocab_list)))
# Parcours de chaque mail dans la liste des index de tous les mails
for i in range(len(all_mails_index)):
    mail_index = all_mails_index[i]
    for index in mail_index:
        X[i, index] = X[i, index] + 1  # Incrémentation du compteur de fréquence du mot correspondant dans le tableau X
        
print(X.shape)
print(y.shape)

(4197, 3740)
(4197,)


In [82]:
# Transformation des données X en les mettant à l'échelle
scaler = StandardScaler()
scaler.fit(X)
scaled_f = scaler.transform(X)


# Étape 2 : Implémentation des modèle


###  1. Reseaux de Neurones

In [110]:
# Diviser les données en ensembles d'apprentissage et de test
X_train, X_test, y_train, y_test = train_test_split(scaled_f, y, test_size=0.2, random_state=42)


In [111]:
# Définition de la taille de l'entrée et de la sortie
#  couche 1 : len mails  *  25
#  couche 2  : 25 * 10
#  couche 3 : 10 * 1

model = MLPClassifier(hidden_layer_sizes=(25, 10))
model.fit(X_train, y_train)

# Faire des prédictions sur de nouvelles données
y_pred = model.predict(X_test)


precision = np.mean(y_test == y_pred) * 100

print("Prsesion : " + str(precision))


Prsesion : 98.09523809523809


### 2. SVM

In [85]:
model= svm.SVC(kernel='linear',probability=True) 
# entrainement 
model.fit(X_train, y_train)

# Faire des prédictions sur de nouvelles données
y_pred = model.predict(X_test)


precision = np.mean(y_test == y_pred) * 100

print("Prsesion : " + str(precision))


Prsesion : 95.47619047619048


###  3. Arbre de desicion

In [86]:
model= tree.DecisionTreeClassifier()
# entrainement 
model.fit(X_train, y_train)

# Faire des prédictions sur de nouvelles données
y_pred = model.predict(X_test)


precision = np.mean(y_test == y_pred) * 100

print("Prsesion : " + str(precision))

Prsesion : 93.57142857142857


###  4. KNN

In [87]:
model= KNeighborsClassifier(n_neighbors=3)
# entrainement 
model.fit(X_train, y_train)

# Faire des prédictions sur de nouvelles données
y_pred = model.predict(X_test)


precision = np.mean(y_test == y_pred) * 100

print("Prsesion : " + str(precision))

Prsesion : 84.76190476190476


In [88]:
model= LogisticRegression(max_iter=1000)
# entrainement 
model.fit(X_train, y_train)

# Faire des prédictions sur de nouvelles données
y_pred = model.predict(X_test)


precision = np.mean(y_test == y_pred) * 100

print("Prsesion : " + str(precision))

Prsesion : 97.61904761904762


## Analyse des résultats

Le tableau suivant contient les résultats des precision calculés sur l'ensemble de test pour chaque algorithme en utilisant le threshold calculé avec la moyenne géométrique de la spécificité et la sensibilité :
   
| Modèle   | RNN         | SVM         | Arbre de desicion | KNN         |  Regression logistique   |
|----------|-------------|-------------|-------------------|-------------|--------------------------|
| Precision | 98.10%     | 97.97%      | 93.10%            | 91.55%      | 97.14%                   |

En utilisant la normalisation nous n'obtenons pas d'améliorations significatives :
| Modèle   | RNN         | SVM         | Arbre de desicion | KNN         |  Regression logistique   |
|----------|-------------|-------------|-------------------|-------------|--------------------------|
| Precision | 97.62%     | 95.48%      | 93.57%            | 84.76%      | 97.62%                   |

## Conclusion  
Les reseaux du neurones est le modèle le plus approprié pour notre classifieur SPAM.